# Chunking your data for RAG

[txtai](https://github.com/neuml/txtai) is an all-in-one embeddings database for semantic search, LLM orchestration and language model workflows.

One of the major workflows in `txtai` is Retrieval Augmented Generation (RAG). Large Language Models (LLM) are built to generate coherent sounding text. While in many cases it is factually accurate, that is not what they're built to do. RAG steps in to help inject smaller pieces of knowledge into a LLM prompt and increase the overall accuracy of responses. The `R` in RAG is very important.

This notebook will demonstrate how to extract, chunk and index text to support retrieval operations for RAG.

# Install dependencies

Install `txtai` and all dependencies.

In [ ]:
%%capture
!pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline-data]

# Data chunking and indexing

Let's dive right in and keep this example simple. The next section creates a [Textractor pipeline](https://neuml.github.io/txtai/pipeline/data/textractor/) and an [Embeddings database](https://neuml.github.io/txtai/embeddings/).

The `Textractor` extracts chunks of text from files and the `Embeddings` takes those chunks and builds an index/database. We'll use a [late chunker](https://docs.chonkie.ai/chunkers/late-chunker) backed by [Chonkie](https://github.com/chonkie-inc/chonkie).

Then, we'll build an indexing workflow that streams chunks from two files.

In [ ]:
from txtai import Embeddings
from txtai.pipeline import Textractor

# Text extraction pipeline with late chunking via Chonkie
textractor = Textractor(chunker="late")
embeddings = Embeddings(content=True)

In [ ]:
def stream():
    urls = ["https://github.com/neuml/txtai", "https://arxiv.org/pdf/2005.11401"]
    for x, url in enumerate(urls):
        chunks = textractor(url)
 
        # Add all chunks - use the same document id for each chunk
        for chunk in chunks:
            yield x, chunk

        # Add the document metadata with the same document id
        # Can be any metadata. Can also be the entire document.
        yield x, {"url": url}

# Index the chunks and metadata
embeddings.index(stream())

A key element of `txtai` that is commonly misunderstood is how to best store chunks of data and join them back to the main document. `txtai` allows re-using the same logical id multiple times.

Behind the scenes, each chunk gets it's own unique index id. The backend database stores chunks in a table called `sections` and data in a table called `documents`. This has been the case as far back as `txtai` 4.0. `txtai` also has the ability to store associated binary data in a table called `objects`. It's important to note that each associated `document` or `object` is only stored once. 

To illustrate, let's look at the first 20 rows in the embeddings database created.

In [63]:
for x in embeddings.search("SELECT indexid, id, url, text from txtai", 20):
    print(x)

{'indexid': 0, 'id': '0', 'url': 'https://github.com/neuml/txtai', 'text': '**GitHub - neuml/txtai: 💡 All-in-one open-source embeddings database for semantic search, LLM orchestration and language model workflows**\n\n*💡 All-in-one open-source embeddings database for semantic search, LLM orchestration and language model workflows - neuml/txtai*\n\n\n\n**All-in-one embeddings database** \ntxtai is an all-in-one embeddings database for semantic search, LLM orchestration and language model workflows.\n\nEmbeddings databases are a union of vector indexes (sparse and dense), graph networks and relational databases.\n\nThis foundation enables vector search and/or serves as a powerful knowledge source for large language model (LLM) applications.\n\nBuild autonomous agents, retrieval augmented generation (RAG) processes, multi-model workflows and more.\n\nSummary of txtai features:\n\n- 🔎 Vector search with SQL, object storage, topic modeling, graph analysis and multimodal indexing\n- 📄 Create

Note that the id/metadata is the same but the indexid and chunk text change with each row.

# Retrieval

Last thing here is to illustrate a couple retrieval operations. LLMs are great at generating answers when we properly bound the context. See the two examples below.

In [61]:
print(embeddings.search("What is it called when LLM generation is bounded with factually correct data?", 1)[0]["text"])

including less of emphasis on lightly editing a retrieved item, but on aggregating content from several
pieces of retrieved content, as well as learning latent retrieval, and retrieving evidence documents
rather than related training pairs. This said, RAG techniques may work well in these settings, and
could represent promising future work.

6 Discussion

In this work, we presented hybrid generation models with access to parametric and non-parametric
memory. We showed that our RAG models obtain state of the art results on open-domain QA. We
found that people prefer RAG’s generation over purely parametric BART, finding RAG more factual
and specific. We conducted an thorough investigation of the learned retrieval component, validating
its effectiveness, and we illustrated how the retrieval index can be hot-swapped to update the model
without requiring any retraining. In future work, it may be fruitful to investigate if the two components
can be jointly pre-trained from scratch, either wi

In [62]:
print(embeddings.search("Tell me about semantic search", 1)[0]["text"])

Traditional search systems use keywords to find data. Semantic search has an understanding of natural language and identifies results that have the same meaning, not necessarily the same keywords.

Get started with the following examples.

|Notebook|Description||
|---|---|---|
|[Introducing txtai](https://github.com/neuml/txtai/blob/master/examples/01_Introducing_txtai.ipynb) |Overview of the functionality provided by txtai||
|[Similarity search with images](https://github.com/neuml/txtai/blob/master/examples/13_Similarity_search_with_images.ipynb) |Embed images and text into the same space for search||
|[Build a QA database](https://github.com/neuml/txtai/blob/master/examples/34_Build_a_QA_database.ipynb) |Question matching with semantic search||
|[Semantic Graphs](https://github.com/neuml/txtai/blob/master/examples/38_Introducing_the_Semantic_Graph.ipynb) |Explore topics, data connectivity and run network analysis||

### LLM Orchestration
Autonomous agents, retrieval augmented genera

Note how both answers give more than enough information for a LLM to answer the question.

# Wrapping up

This notebook covered how to build a retrieval system for RAG with txtai. Chunking and retrieval are key pieces of a RAG system, arguably the most important. With the commoditization of LLMs, it's going to be more and more important on how data is presented to LLMs. When given concise information, LLMs can take it from there!
